In [ ]:
import pandas as pd
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv(r'Data/Itau.csv')

In [ ]:
def encontrar_min_analistas():
    min_analistas = 1
    max_analistas = 150
    melhor_solucao = None

    while min_analistas <= max_analistas:
        mid = (min_analistas + max_analistas) // 2
        print(f"Tentando com {mid} analistas...")
        status, solucao = resolver_alocacao(mid)

        if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
            melhor_solucao = solucao
            max_analistas = mid - 1  # tenta reduzir ainda mais
        else:
            min_analistas = mid + 1  # precisa de mais analistas

    return melhor_solucao


def resolver_alocacao(MAX_ANALISTAS):
    model = cp_model.CpModel()

    HORAS = list(range(8, 18))
    DEMANDA = [215, 386, 275, 315, 145, 257, 216, 185, 199, 285]
    TMA_EM_SEGUNDOS = 156
    PROPOSTAS_POR_HORA = int(3600 / TMA_EM_SEGUNDOS)
    HORAS_TRABALHADAS = 9

    x = {}
    almoco = {}

    for a in range(MAX_ANALISTAS):
        for h in HORAS:
            x[a, h] = model.NewBoolVar(f"x_{a}_{h}")
        for h in [12, 13, 14]:
            almoco[a, h] = model.NewBoolVar(f"almoco_{a}_{h}")

    for a in range(MAX_ANALISTAS):
        model.Add(sum(x[a, h] for h in HORAS) == HORAS_TRABALHADAS)
        model.Add(sum(almoco[a, h] for h in [12, 13, 14]) == 1)
        for h in [12, 13, 14]:
            model.Add(x[a, h] == 0).OnlyEnforceIf(almoco[a, h])

    for i, h in enumerate(HORAS):
        model.Add(sum(x[a, h] for a in range(MAX_ANALISTAS)) * PROPOSTAS_POR_HORA >= DEMANDA[i])

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 30.0  # mais leve para cada rodada
    status = solver.Solve(model)

    solucao = []
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        for a in range(MAX_ANALISTAS):
            horas = [h for h in HORAS if solver.Value(x[a, h])]
            if horas:
                almoco_h = next((h for h in [12, 13, 14] if solver.Value(almoco[a, h])), None)
                solucao.append((a, horas, almoco_h))
    return status, solucao

In [ ]:
solucao_otima = encontrar_min_analistas()

if solucao_otima:
    for a, horas, almoco_h in solucao_otima:
        print(f"Analista {a:2d} | Trabalha: {horas} | Almoço: {almoco_h}:00")
else:
    print("❌ Nenhuma solução encontrada.")

In [ ]:
def resolver_alocacao_dinamica(streamlit):
    # Dados de entrada
    intervalos = streamlit.dataframe_sla['horario'].tolist()
    demandas = streamlit.dataframe_sla['quantidade'].tolist()
    tma = streamlit.tma  # em segundos
    
    # Calcular duração de cada intervalo em minutos
    duracoes = []
    for ts in intervalos:
        # Converter timestamp para minutos do dia
        total_minutos = ts.hour * 60 + ts.minute
        duracoes.append(total_minutos)
    
    n_intervalos = len(intervalos)
    
    # Duração total do turno (9h48 = 588 minutos)
    DURACAO_TURNO = 588
    # Janela de almoço (11:00-14:00) em minutos
    ALMOCO_INICIO_MIN = 11 * 60  # 660 minutos
    ALMOCO_FIM_MIN = 14 * 60      # 840 minutos
    
    # Criar modelo
    model = cp_model.CpModel()
    MAX_ANALISTAS = 30  # Número máximo de analistas (ajustável)

    # Fator de escala para converter floats em inteiros
    FATOR_ESCALA = 1000

    # Variáveis
    entrada = []  # Minuto de entrada (0-1440)
    almoco_inicio = []  # Minuto de início do almoço
    analista_ativo = []  # Se o analista é utilizado
    # Para cada analista e cada intervalo, se está trabalhando
    trabalhando = [[model.NewBoolVar(f'trab_a{a}_i{i}') 
                   for i in range(n_intervalos)] 
                   for a in range(MAX_ANALISTAS)]

    for a in range(MAX_ANALISTAS):
        # Entrada entre 7:00 e 9:00 (420-540 minutos)
        entrada.append(model.NewIntVar(420, 540, f'entrada_{a}'))
        # Almoço entre 11:00 e 14:00 (660-840 minutos)
        almoco_inicio.append(model.NewIntVar(660, 840, f'almoco_{a}'))
        analista_ativo.append(model.NewBoolVar(f'ativo_{a}'))

    # Restrições para cada analista
    for a in range(MAX_ANALISTAS):
        # Fim do turno (entrada + duração)
        fim_turno = model.NewIntVar(0, 24*60, f'fim_turno_{a}')
        model.Add(fim_turno == entrada[a] + DURACAO_TURNO)
        
        # Fim do almoço
        fim_almoco = model.NewIntVar(0, 24*60, f'fim_almoco_{a}')
        model.Add(fim_almoco == almoco_inicio[a] + 60)
        
        # Restrição: almoço dentro do turno
        model.Add(almoco_inicio[a] >= entrada[a])
        model.Add(fim_almoco <= fim_turno)
        
        # Para cada intervalo
        for i in range(n_intervalos):
            inicio_intervalo = duracoes[i]
            fim_intervalo = inicio_intervalo + 15  # Intervalo de 15 minutos
            
            # O analista só pode trabalhar se:
            # 1. Estiver ativo
            # 2. O intervalo estiver dentro do turno
            # 3. Não estiver no horário de almoço
            
            # Variável auxiliar para intervalo dentro do turno
            dentro_turno = model.NewBoolVar(f'dentro_turno_a{a}_i{i}')
            model.Add(entrada[a] <= inicio_intervalo).OnlyEnforceIf(dentro_turno)
            model.Add(fim_turno >= fim_intervalo).OnlyEnforceIf(dentro_turno)
            model.AddBoolOr([dentro_turno.Not(), 
                            model.NewConstant(1)])  # Forçar definição
            
            # Variável auxiliar para fora do almoço
            fora_almoco = model.NewBoolVar(f'fora_almoco_a{a}_i{i}')
            model.Add(almoco_inicio[a] >= fim_intervalo).OnlyEnforceIf(fora_almoco)
            model.Add(fim_almoco <= inicio_intervalo).OnlyEnforceIf(fora_almoco)
            model.AddBoolOr([fora_almoco.Not(), 
                            model.NewConstant(1)])  # Forçar definição
            
            # Combinar condições
            model.AddBoolAnd([dentro_turno, fora_almoco, analista_ativo[a]]).OnlyEnforceIf(trabalhando[a][i])
            model.AddBoolOr([
                trabalhando[a][i].Not(),
                dentro_turno,
                fora_almoco,
                analista_ativo[a]
            ])

    # Satisfazer demanda para cada intervalo
    for i in range(n_intervalos):
        # Calcular capacidade necessária por minuto
        capacidade_por_minuto = 60 / tma  # propostas por minuto
        
        # Capacidade de um analista neste intervalo (15 minutos)
        capacidade_analista = 15 * capacidade_por_minuto
        
        # Converter para inteiro usando fator de escala
        capacidade_analista_int = int(round(capacidade_analista * FATOR_ESCALA))
        demanda_int = int(round(demandas[i] * FATOR_ESCALA))
        
        # Total de capacidade para o intervalo (como inteiro)
        capacidade_total = sum(
            trabalhando[a][i] * capacidade_analista_int 
            for a in range(MAX_ANALISTAS)
        )
        
        # Criar variável para a soma
        cap_total_var = model.NewIntVar(0, MAX_ANALISTAS * capacidade_analista_int, f'cap_total_i{i}')
        model.Add(cap_total_var == capacidade_total)
        model.Add(cap_total_var >= demanda_int)

    # Minimizar número de analistas
    model.Minimize(sum(analista_ativo))

    # Resolver
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 120.0
    status = solver.Solve(model)

    # Processar resultados
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        num_analistas = int(solver.ObjectiveValue())
        alocacoes = []
        
        for a in range(MAX_ANALISTAS):
            if solver.Value(analista_ativo[a]):
                entrada_min = solver.Value(entrada[a])
                almoco_min = solver.Value(almoco_inicio[a])
                
                # Converter minutos para horário
                entrada_hr = f"{entrada_min//60:02d}:{entrada_min%60:02d}"
                almoco_hr = f"{almoco_min//60:02d}:{almoco_min%60:02d}"
                saida_hr = f"{(entrada_min + DURACAO_TURNO)//60:02d}:{(entrada_min + DURACAO_TURNO)%60:02d}"
                
                alocacoes.append({
                    'entrada': entrada_hr,
                    'almoco': almoco_hr,
                    'saida': saida_hr
                })
        
        return alocacoes, num_analistas
    else:
        return None, 0

# Função principal permanece igual
def athena(streamlit, calculadora):
    # Resolver alocação
    alocacoes, num_analistas = resolver_alocacao_dinamica(streamlit)
    
    if alocacoes is None:
        print("❌ Nenhuma solução encontrada")
        return []
    
    print(f"🔢 Analistas usados: {num_analistas}")
    
    # Criar e retornar objetos Analista
    analistas = []
    for aloc in alocacoes:
        analista = Analista(
            tma=streamlit.tma,
            entrada=aloc['entrada'],
            almoco=aloc['almoco'],
            saida=aloc['saida']
        )
        analistas.append(analista)
    
    return analistas